In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import time
import gurobipy as gp
from gurobipy import GRB
from statistics import mean
from tqdm import tqdm
from test_functions import n_num_test

#Todo
"""

"""

"""

Here we plot the computation time of SMD and FMD for different n values. 

Update Notes: 
Fixed plotting codes.


"""


current_path = os.getcwd()
parent_path = os.path.dirname(current_path)

J_nt = 10  # Number of Cohorts
L_nt = 50 # Number of Treatments

n_list_nt = [20000,12000]
#n_list_nt = np.linspace(10000,20000,11)
m_nt = 5  # constraint number for figure 1
repeats_nt = 2

data_list = []

#J_nt x L_nt
for n_num in range(len(n_list_nt)):
    temp_data_list = [n_list_nt[n_num], J_nt * L_nt, m_nt]
    SMD_run_time_arr = np.zeros(repeats_nt)
    FMD_run_time_arr = np.zeros(repeats_nt)
    SMD_real_T_arr = np.zeros(repeats_nt)
    FMD_real_T_arr = np.zeros(repeats_nt)
    SMD_T_arr = np.zeros(repeats_nt)
    FMD_T_arr = np.zeros(repeats_nt)

    for rep_idx in range(repeats_nt):
        stat_SMD, stat_FMD, _ = n_num_test(int(n_list_nt[n_num]), J_nt, L_nt, m_nt, print_opt = 1, feas_opt = 1)
        SMD_run_time_arr[rep_idx] = stat_SMD.total_solved_time
        FMD_run_time_arr[rep_idx] = stat_FMD.total_solved_time
        SMD_real_T_arr[rep_idx] = stat_SMD.real_T_list[0]
        FMD_real_T_arr[rep_idx] = stat_FMD.real_T_list[0]
        SMD_T_arr[rep_idx] = stat_SMD.max_iter
        FMD_T_arr[rep_idx] = stat_FMD.max_iter
    temp_data_list.append(np.mean(SMD_run_time_arr))
    temp_data_list.append(np.mean(SMD_real_T_arr))
    temp_data_list.append(np.mean(SMD_T_arr))
    temp_data_list.append(np.mean(FMD_run_time_arr))
    temp_data_list.append(np.mean(FMD_real_T_arr))
    temp_data_list.append(np.mean(FMD_T_arr))
    data_list.append(temp_data_list)

df= pd.DataFrame(data_list, columns = ['n','d','m','SMD_Avg_Solve_time', 'SMD_Avg_Iter', 'SMD_Max_Iter',\
                'FMD_Avg_Solve_time', 'FMD_Avg_Iter', 'FMD_Max_Iter'])
save_path = parent_path + '/results/n_test_d=' + str(J_nt * L_nt) + '_m=' + str(m_nt) + '.csv'
df.to_csv(save_path, index=False)

In [ ]:
"""

Plot x-axis: n_num, y-axis: average time

"""

"""
#Create random mean matrix for our data, [0,0.25] uniform distribution
mean_array_sst = np.random.rand(m_nt, J_nt,L_nt) / J_nt

# stat1_list[n_idx][repeat_idx][alg_idx]
for n_num in n_list_nt:

    stat_repeat = []

    # Create our dataset
    emp_dist_value = np.zeros([m_nt, J_nt, L_nt, n_num])
    emp_dist_value[:] = np.nan
    x_0_nt = np.ones([J_nt, L_nt]) / L_nt
    p_0_nt = np.ones([m_nt, n_num]) / n_num
    RHS_nt = np.ones([m_nt])




#     emp_dist_value_1 = np.random.normal(loc=0.5, scale=0.125, size=(m_nt, J_nt, int(L_nt/2), n_num))
#     emp_dist_value_2 = np.random.normal(loc=0.25, scale=0.125, size=(m_nt, J_nt, int((L_nt+1)/2), n_num))
#     emp_dist_value = np.concatenate([emp_dist_value_1, emp_dist_value_2], axis=2)

    for rep_idx in tqdm(range(repeats_nt)):

        print('Problem n = %s, m = %s, J = %s, L = %s' %(n_num, m_nt,J_nt,L_nt))

        #Random mean data generation
        emp_dist_value = np.zeros([m_nt,J_nt,L_nt,n_num])
        emp_dist_value[:] = np.nan
        for m_idx in range(m_nt):
            for j_idx in range(J_nt):
                for l_idx in range(L_nt):
                    emp_dist_value[m_idx,j_idx,l_idx,:] = np.random.normal(loc = mean_array_sst[m_idx,j_idx, l_idx],\
                         scale= var_scale_nt * mean_array_sst[m_idx,j_idx, l_idx], size = n_num)




        G_nt = np.absolute(emp_dist_value).max()
        print('G:', G_nt)
        M_nt = []
        for i in range(m_nt):
            M_nt.append(J_nt * np.absolute(emp_dist_value[i, :, :, :]).max())



        stat_alg = []

        for alg_idx in range(len(alg_list_nt)):
            if alg_list_nt[alg_idx][0] == 'i_star':
                K_nt = n_num
            else:
                K_nt = alg_list_nt[alg_idx][0]

            if alg_list_nt[alg_idx][1] == 'FGD':
                K_grad_nt = n_num
            else:
                K_grad_nt = alg_list_nt[alg_idx][1]

            if K_nt == n_num:
                C_K_nt = 1

            # Calculate stoc_factor if K_grad < n

            # Omega = np.log(2 * m_1 / nu)
            #
            # if K_grad < n_num:
            #     stoc_factor = 1 + Omega / 2 + 4 * math.sqrt(Omega)

            stoc_factor_nt = 1

            if ss_type_nt == 'constant':
                T_nt, R_x, R_p, ss_x, ss_p = R_x_p_combined_constant(J_nt, L_nt, n_num, G_nt, M_nt, delta_nt, rho_nt,\
                                                        obj_tol_nt,C_K_nt, stoc_factor_nt, K_grad_nt)
            elif ss_type_nt == 'diminish':
                T_nt, R_x, R_p, ss_x, ss_p = R_x_p_combined_diminish(J_nt, L_nt, n_num, G_nt, M_nt, delta_nt, rho_nt,\
                                                        obj_tol_nt,C_K_nt, stoc_factor_nt, K_grad_nt)
            print("Max Iteration:", T_nt)
            # Generate random_samples_list
            random_samples_list = np.random.rand(T_nt, m_nt, K_nt)

            alg_stat = DRO_Solver(x_0_nt, p_0_nt, emp_dist_value,\
                                      K_nt, K_grad_nt, 'entropy', 'chi-square', delta_nt, rho_nt, nu_nt, alpha_tol_nt,\
                                      opt_low_nt, opt_up_nt, obj_tol_nt,
                                      RHS_nt, ss_type_nt, \
                                      random_samples_list, C_K_nt, dual_gap_freq_nt,T_cap_nt,\
                                      print_option=1, K_test_flag= 0, min_flag=0)

            stat_alg.append(alg_stat)
        stat_repeat.append(stat_alg)
    stat_list_nt.append(stat_repeat)

if len(alg_list_nt) == 2:
    marker_list = ['o', 'd']
    color_list = ['b', 'g']
    ls_list = ['-', '--']  # line Style
    alg_name = ['Stoch Subgrad', 'Full Subgrad']
"""

# Draw Plots

# Change the order of the list first
# Also calculate average solved time
# avg_runtime_list_nt = []  # total_run_time_list[alg_idx][n_idx]
#
# # In[ ]:
#
#
# for alg_idx in range(2):
#     temp_list = []
#
#     for n_idx in range(len(n_list_nt)):
#         temp = 0
#
#         for rep_idx in range(repeats_nt):
#             temp += stat_list_nt_diminish[n_idx][rep_idx][alg_idx].total_solved_time
#         temp /= repeats_nt
#         temp_list.append(temp)
#
#     avg_runtime_list_nt.append(temp_list)
#
# alg_name = ['FMD Dim', 'SMD Dim']
#
# for alg_idx in range(len(avg_runtime_list_nt)):
#     plt.plot(n_list_nt, avg_runtime_list_nt[alg_idx], label=alg_name[alg_idx])
#     plt.xlabel('n')
#     plt.ylabel('time(s)')
#     plt.title('Average Solve Times for Different n (m=%s)' % m_nt)
#     plt.legend(loc="upper left")
#
#
# save_path = parent_path + '/results/nt_epsilon_' + str(obj_tol_nt) + '.png'
#
# plt.savefig(save_path)
#
# plt.show()